In [1]:
# !pip install bitsandbytes
# !pip install peft
!pip install scipy
!pip install -q  torch peft==0.4.0 bitsandbytes==0.40.2 transformers==4.30 trl==0.4.7 accelerate sentencepiece


In [2]:
import argparse
import bitsandbytes as bnb
from datasets import load_dataset
from functools import partial
import os
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, AutoPeftModelForCausalLM
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, set_seed, Trainer, TrainingArguments, BitsAndBytesConfig, \
    DataCollatorForLanguageModeling, Trainer, TrainingArguments

In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


In [4]:
prompt_data = pd.read_csv("mini_data.csv")[["Question_Text", "Answer_Text"]]

In [5]:

import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline
)
from peft import LoraConfig
from trl import SFTTrainer

In [6]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /roo

In [8]:
training_data = prompt_data

# Model and tokenizer names
#base_model_name = "huggyllama/llama-7b"#"TheBloke/Llama-2-7B-chat-GGML" # "meta-llama/Llama-2-7b-hf" #"NousResearch/Llama-2-7b-chat-hf"
base_model_name = "meta-llama/Llama-2-7b-hf"
refined_model = "llama-2-7b-heartful-enhanced" #You can give it your own name

# Tokenizer
#llama_tokenizer = AutoTokenizer.from_pretrained(base_model_name , use_fast=False)
llama_tokenizer = AutoTokenizer.from_pretrained(base_model_name, use_fast=False)
llama_tokenizer.pad_token = llama_tokenizer.eos_token
llama_tokenizer.padding_side = "right"  # Fix for fp16

# Quantization Config
quant_config = BitsAndBytesConfig(
    load_in_4bit=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False
)


In [9]:
#pip install transformers==4.30
import torch
!pip install einops

In [10]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
#!pip install torchvision==0.11.2+cu113 torchaudio===0.10.1+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html --quiet

Looking in indexes: https://download.pytorch.org/whl/cu118


In [11]:
# Model
from accelerate import init_empty_weights
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    quantization_config=quant_config,
    device_map={"": 0},
    load_in_4bit=True
)
base_model.config.use_cache = False
base_model.config.pretraining_tp = 1

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

OutOfMemoryError: ignored

In [12]:
import torch
torch.cuda.is_available()

True